# Chapter 9. 추천 시스템

## 9.1 추천 시스템의 개요와 배경
하나의 컨텐츠를 선택했을 때 선택된 콘텐츠와 연관된 추천 콘텐츠가 얼마나 사용자의 관심을 끌고 개인에게 맞춘 콘텐츠를 추천했는지가 사이트의 평판을 좌우하는 중요 요소. 더 많은 데이터가 추천 시스템에 축적되며 추천이 더욱 정확해지고 다양한 결과를 얻을 수 있는 좋은 선순환 시스템 구축 가능.

### 추천 시스템의 유형
- 콘텐츠 기반 필터링 방식
- 협업 필터링 방식
  - 최근접 이웃 협업 필터링
  - 잠재 요인 협업 필터링

## 9.2 콘텐츠 기반 필터링 추천 시스템
사용자가 특정 아이템을 매우 선호하는 경우 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식.

## 9.3 최근접 이웃 협업 필터링
사용자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은 사용자 행동 양식만을 기반으로 추천을 수행하는 것이 협업 필터링 방식. 주요 목표는 사용자-아이템 평점 매트릭스 같은 축적된 사용자 행동 데이터를 기반으로 사용자가 아직 평가하지 않은 아이템을 예측 평가하기.
- 사용자-아이템 행렬
  - 행: 개별 사용자
  - 열: 개별 아이템
  - 값: 평점
  - 희소 행렬 특성: 사용자가 아이템에 대한 평점을 매기는 경우가 많지 않음.
  - 사용자 기반: 당신과 비슷한 고객이 다음 상품도 구매
  - 아이템 기반: 이 상품을 선택한 다른 고객들은 다음 상품도 구매
  

## 9.4 잠재 요인 협업 필터링
사용자-아이템 평점 매특릭스 속에 숨어 있는 잠재 요인을 추출해 추천 예측을 할 수 있게 하는 기법. 대규모 다차원 행렬을 차원 감소 기법으로 분해하는 과정에서 잠재 요인 추출. 행렬 분해 기반의 잠재 요인 협업 필터링.
- 행렬 분해는 다차원 매트릭스를 저차원 매트릭스로 분해하는 기법: SVD, NMF
- 확률적 경사 하강법을 이용한 행렬 분해: P와 Q 행렬 값이 실제 R 행렬 값과 가장 최소 오류를 가질 수 있도록 반복적인 비용함수 최적화를 통해 P와 Q 유추

In [1]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재 요인 차원 K는 3으로 설정.
R = np.array([[4,np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3,1],
              [np.NaN, np.NaN, 3,4,4],
              [5,2,1,2,np.NaN]])
num_users, num_items = R.shape
K=3

# P와 Q 행렬의 크기를 지정하고 정규 분포를 가진 임의의 값 입력
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users,K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [2]:
from sklearn.metrics import mean_squared_error

def get_rmse(R,P,Q, non_zeros):
  error=0
  # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
  full_pred_matrix = np.dot(P, Q.T)

  # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출해  실제 R 행렬과 예측 행렬의 RMSE 추출
  x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
  y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
  R_non_zeros = R[x_non_zero_ind,y_non_zero_ind]
  full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
  mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
  rmse = np.sqrt(mse)

  return mse

In [4]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] >0]

steps=1000
learning_rate=0.01
r_lambda = 0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
for step in range(steps):
  for i,j,r in non_zeros:
    # 실제 값과 예측 값의 차이인 오류 값 구함
    eij = r-np.dot(P[i,:], Q[j,:].T)
    # Regularization 을 반영한 SGD 업데이트 공식 적용
    P[i,:] = P[i,:]+learning_rate*(eij*Q[j,:]-r_lambda*P[i,:])
    Q[i,:] = Q[i,:]+learning_rate*(eij*P[i,:]-r_lambda*Q[j,:])
  rmse = get_rmse(R,P,Q, non_zeros)
  if (step%50) ==0:
    print('### iteration step :', step, "rmse:",rmse)

### iteration step : 0 rmse: 10.385304998472746
### iteration step : 50 rmse: 3.4054251934930844
### iteration step : 100 rmse: 3.296956174050893
### iteration step : 150 rmse: 3.460010472458482
### iteration step : 200 rmse: 3.4361312112964906
### iteration step : 250 rmse: 3.3799122109522712
### iteration step : 300 rmse: 3.2557072167194545
### iteration step : 350 rmse: 3.5412681008065228
### iteration step : 400 rmse: 3.558318516212212
### iteration step : 450 rmse: 3.547419879523234
### iteration step : 500 rmse: 3.5486141207464112
### iteration step : 550 rmse: 3.591298399222483
### iteration step : 600 rmse: 3.6682629115986582
### iteration step : 650 rmse: 3.724510829115083
### iteration step : 700 rmse: 3.669879397072798
### iteration step : 750 rmse: 3.236106361924676
### iteration step : 800 rmse: 3.7484620137730134
### iteration step : 850 rmse: 3.694496353859975
### iteration step : 900 rmse: 3.7499125716370956
### iteration step : 950 rmse: 3.8809092211163896


In [5]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix,3))

예측 행렬:
 [[  3.97  -17.306  16.888   2.01    0.489]
 [ -0.498   4.969  -1.816   3.045   0.046]
 [  0.784   0.249   3.112   4.235   0.233]
 [  0.245  -0.091  -1.41   -0.712   0.049]]
